In [5]:
import pandas as pd
import pyodbc
import ibm_db as db
import ibm_db_dbi
from tqdm import tqdm
import warnings

def pull_snp_member(start_date, end_date, addl_cols = []):
    conn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                        'Server=MDCRBIPRODDB;'
                        'Database=StarsBIDataProd;'
                        'Trusted_Connection=yes;')
    
    select_list = ["MEDICARE_NUMBER", "Member_ID", "SNP", "cast(Eff_Date as varchar(10)) as Eff_Date",
                    "cast (Term_Date as varchar(10)) as Term_Date","Contract_Number","PBP","CMS_State",
                    "CMS_County","Age", "Gender", "FIRST_NAME", "LAST_NAME", "MIDDLE_INITIAL", "Address1",
                    "Address2", "City", "State", "left(Zip,5) as Zip", "MEM_Phone", "Writing_Agent_Name",
                    "Recruiter_Name", "TOH_Name", "Sales_Channel", "PCP_Name", "PCP_Tax_ID", "Enroll_Status",
                    "DRC_Term_Status","Movement_Type_Out"]
    if len(addl_cols) >= 1:
        select_list = select_list + addl_cols

    query = """select distinct top 5 """\
          + ', '.join(select_list) +\
        """
        from DM.MSBI_MEPR 
        where PRODUCT='MA' 
        and ISSUED_STATUS = 'Issued' 
        and Group_Status='Individual'
        and Term_Date >= ?
        and Eff_Date <= ?
        """

    row_count = pd.read_sql_query('select count(*) from ({}) subquery'.format(query),conn, params=[start_date, end_date]).iloc[0,0]
    dfs =[]
    with tqdm(total=row_count) as pbar:
        for chunk in pd.read_sql(query, conn, params=[start_date, end_date], chunksize = 1000):
            dfs.append(chunk)
            pbar.update(len(chunk))
    df = pd.concat(dfs, ignore_index = True)
    return df

In [11]:
test_df = pull_snp_member(start_date="2023-11-01", end_date = "2023-11-10")

C:\Users\A845514\AppData\Local\Temp\ipykernel_28340\3066506595.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  row_count = pd.read_sql_query('select count(*) from ({}) subquery'.format(query),conn, params=[start_date, end_date]).iloc[0,0]
  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\A845514\AppData\Local\Temp\ipykernel_28340\3066506595.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.read_sql(query, conn, params=[start_date, end_date], chunksize = 1000):
100%|██████████| 5/5 [00:02<00:00,  1.99it/s]


In [9]:
test_df.head()

,MEDICARE_NUMBER,Member_ID,SNP,Eff_Date,Term_Date,Contract_Number,PBP,CMS_State,CMS_County,Age,...,MEM_Phone,Writing_Agent_Name,Recruiter_Name,TOH_Name,Sales_Channel,PCP_Name,PCP_Tax_ID,Enroll_Status,DRC_Term_Status,Movement_Type_Out
0,7GC4P88NC81,101337328200,N,2023-01-01,9999-12-31,H5521,127,WA,KING,68.0,...,4258684473,None,None,None,CMSOEC,King County Public Hospital District No. 2,910844563,Active,None,Active
1,9TT8JJ2AP24,101487685100,N,2022-01-01,9999-12-31,H1609,001,IA,BREMER,76.0,...,3195722196,"SAINSBURY, LISA",Extend Insurance Services LLC,EXTEND INSURANCE SERVICES LLC,EmpGroupExchange,Waverly Health Center,421029188,Active,None,Active
2,7AM6NY3XR41,101332664000,N,2022-01-01,2023-12-31,H5521,269,FL,SAINT JOHNS,71.0,...,9044039439,"Jansen, Britta","BRITKEN ENTERPRISES, INC. DBA MODERN SENIOR HE...",INTEGRITY MARKETING GROUP,NDP,Baptist Primary Care Inc,593647972,Term,Unidentified,Migration
3,7F92DJ4QU12,101594604200,N,2023-01-01,9999-12-31,H0523,073,CA,ORANGE,69.0,...,7143252541,"MARTINEZ, LIBERATO","GOLDEN OUTLOOK, INC.","HEALTHMARKETS INSURANCE AGENCY, INC.",NDP,MemorialCare Medical Foundation,271504911,Active,None,Active
4,6VT8YX0JD55,101042742200,N,2022-01-01,2023-12-31,H7149,001,NE,LANCASTER,79.0,...,3088821435,"Schueth, James",Saving Seniors Money Inc,INTEGRITY MARKETING GROUP,NDP,Bryan Physician Network,201357375,Term,Unidentified,Plan_Change


In [8]:
test_df2 = pull_snp_member(start_date="2023-11-01", end_date = "2023-11-10", addl_cols = ["BatchID", "TOH_ID"])

C:\Users\A845514\AppData\Local\Temp\ipykernel_28340\3066506595.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  row_count = pd.read_sql_query('select count(*) from ({}) subquery'.format(query),conn, params=[start_date, end_date]).iloc[0,0]
  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\A845514\AppData\Local\Temp\ipykernel_28340\3066506595.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.read_sql(query, conn, params=[start_date, end_date], chunksize = 1000):
100%|██████████| 5/5 [00:02<00:00,  1.97it/s]


In [10]:
test_df2.head()

,MEDICARE_NUMBER,Member_ID,SNP,Eff_Date,Term_Date,Contract_Number,PBP,CMS_State,CMS_County,Age,...,Recruiter_Name,TOH_Name,Sales_Channel,PCP_Name,PCP_Tax_ID,Enroll_Status,DRC_Term_Status,Movement_Type_Out,BatchID,TOH_ID
0,7GC4P88NC81,101337328200,N,2023-01-01,9999-12-31,H5521,127,WA,KING,68.0,...,None,None,CMSOEC,King County Public Hospital District No. 2,910844563,Active,None,Active,None,None
1,9TT8JJ2AP24,101487685100,N,2022-01-01,9999-12-31,H1609,001,IA,BREMER,76.0,...,Extend Insurance Services LLC,EXTEND INSURANCE SERVICES LLC,EmpGroupExchange,Waverly Health Center,421029188,Active,None,Active,None,123003
2,7AM6NY3XR41,101332664000,N,2022-01-01,2023-12-31,H5521,269,FL,SAINT JOHNS,71.0,...,"BRITKEN ENTERPRISES, INC. DBA MODERN SENIOR HE...",INTEGRITY MARKETING GROUP,NDP,Baptist Primary Care Inc,593647972,Term,Unidentified,Migration,None,283186
3,7F92DJ4QU12,101594604200,N,2023-01-01,9999-12-31,H0523,073,CA,ORANGE,69.0,...,"GOLDEN OUTLOOK, INC.","HEALTHMARKETS INSURANCE AGENCY, INC.",NDP,MemorialCare Medical Foundation,271504911,Active,None,Active,None,187404
4,6VT8YX0JD55,101042742200,N,2022-01-01,2023-12-31,H7149,001,NE,LANCASTER,79.0,...,Saving Seniors Money Inc,INTEGRITY MARKETING GROUP,NDP,Bryan Physician Network,201357375,Term,Unidentified,Plan_Change,None,283186
